In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
learning_rate = 3e-2
max_iters = 1000
eval_iters = 250

cuda:0


In [9]:
# with open('wizard_of_oz.txt', 'r', encoding = 'utf-8') as f:
#     text = f.read()
# chars = sorted(set(text))

# vocab_size = len(chars)

x = torch.empty(5,3,4,1)
y = torch.empty(1,4,1)
(x.add_(y)).size()

torch.Size([5, 3, 4, 1])

string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype= torch.long)
print(data[:100])

In [4]:
n = int(0.8 *len(data))
train_data = data[:n]
val_data = data[n:]



In [5]:
block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
   
    print('when input is: ', context, 'target is ', target)
    
print(context)
print(target)

when input is:  tensor([75]) target is  tensor(27)
when input is:  tensor([75, 27]) target is  tensor(63)
when input is:  tensor([75, 27, 63]) target is  tensor(66)
when input is:  tensor([75, 27, 63, 66]) target is  tensor(63)
when input is:  tensor([75, 27, 63, 66, 63]) target is  tensor(68)
when input is:  tensor([75, 27, 63, 66, 63, 68]) target is  tensor(56)
when input is:  tensor([75, 27, 63, 66, 63, 68, 56]) target is  tensor(73)
when input is:  tensor([75, 27, 63, 66, 63, 68, 56, 73]) target is  tensor(1)
tensor([75, 27, 63, 66, 63, 68, 56, 73])
tensor(1)


In [14]:
@torch.no_grad()

def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [15]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
       
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
           
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)





context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(model.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


OWKWMbEKP
60Vm4LSkdLCroHnvfhP?s8pii9﻿tshLGY0ciJq;OWyyY G067mzYKzS;oP9?
yaFQ7C0aSxw
WH7&L4yzPb3y6E!F,
'Zs:wCIqh3Lt,3QINO3z0Vi)8iYV'eVt0VNMD8
c3sPPEe2lf. 5!gZT.AFl&Lr14uUv5Sg4﻿'Vn0iuhab)mLBZ(cQ!ethSd&sniNz9PL4ymf﻿m9T5h5uMw
e)vpIn2GP(xT2wPbynV.:
UkfGoOpH2cpVn5e:A2k2;:tKJst!1iEaVL&oOU
8D6bNcm
A
j3n8kS64oR.Aw
vT;qKW﻿wy9em.7VH!W﻿
,f﻿W﻿9RU8Tvp?t9:2TwFQ
a8
ODax
Bs?OiJ3?MB1i9c﻿ Db2c1wIpbcOW9 zA,bkRbW
cYIBqYO5(LFhPPht98!H;L4ABSR8p'﻿"Qz9F.lHiOBNI4uMFcm.7NmvpAbrHSwACxg'Q7nzSiuiGCH0V-C Nt,HrABZ&FQL&WcsuiGhNT


In [16]:
def get_batch(split):
    data = train_data if split =='train' else val_data
    # This generates random indexes to pull 8 elements from.
    ix = torch.randint(len(data) - block_size, (batch_size,))
   
    # take tensors from 30233 to 30241(30233+8) and do this for
    # every tensor in ix.
    x = torch.stack([data[i:i+block_size] for i in ix])
    #take tensors from 30234(30233+1) to 30242(30233+8+1) and do this 
    # for every tensor in ix.
    y = torch.stack([data[i+1:i+block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y

x,y = get_batch('train')
print(x.device)
print(y.device)
print('inputs:')
print(x)
print('targets:')
print(y)
                       

cuda:0
cuda:0
inputs:
tensor([[56, 57, 67,  1, 60, 49, 62, 52],
        [ 1, 68, 63,  1, 68, 56, 53,  1],
        [56, 73,  8,  1, 49, 62, 52,  1],
        [68, 56, 53,  1, 63, 68, 56, 53]], device='cuda:0')
targets:
tensor([[57, 67,  1, 60, 49, 62, 52,  1],
        [68, 63,  1, 68, 56, 53,  1, 53],
        [73,  8,  1, 49, 62, 52,  1, 68],
        [56, 53,  1, 63, 68, 56, 53, 66]], device='cuda:0')


In [17]:
print(len(generated_chars))

501


In [44]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'step: {iter},train loss: {losses["train"]:.3f}, val loss: {losses["val"]:.3f}')

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss

    logits, loss = model.forward(xb, yb)
   
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0,train loss: 2.423, val loss: 2.473
step: 250,train loss: 2.443, val loss: 2.545
step: 500,train loss: 2.433, val loss: 2.484
step: 750,train loss: 2.445, val loss: 2.533
2.3225643634796143


In [45]:
context = torch.zeros((1,1), dtype = torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens = 500)[0].tolist())
print(generated_chars)


Fon oond towond Malled proystheatcrathered "Thy, aro aindled t t spowhid firgimethe the defer. hy y air."Bkeronen, Dowimy we,  th s o hawlor isthe plyosun t tche farpl inoror y tame inoude t hovete
or warode, rad d omeat an me
ted," se
"hel Zey g wads, the atid alf  Doke I e Houthend thegr ofe evem," to wlk s Don athert d calon, Wit aitroroous fe ngis.'sizas
ttele ill thathenlfird til t thous nd, t's e h oomy, plararom  he Ozar I wan I o e thealdsmely cocend Wicl!" th.

"Wil santharouts the

can
